In [4]:
from scipy.spatial.distance import cdist
import pandas as pd
import numpy as np

from src.objects.customer import Customer
from src.aco.aco import ACO
import src.variables as vars
import src.printer as pr
import random

In [5]:
def instance_generator(customer_count, rand_seed=0):
    """
    Generates a random instance of the vehicle routing problem.

    Returns:
    tuple: List of customer objects and the depot object., number of trucks, capacity of each truck
    """
    random.seed(rand_seed)  # Set the random seed for reproducibility
    
    # Number of trucks
    truck_count = random.randint(10, 75)
    
    # Capacity of each truck
    capacity = random.randint(10, 50) * 10

    # Number of customers
    num_customers = customer_count
    
    # Generate random service time for the customer
    service_time = random.randint(1, 5) * 10

    for i in range(1, num_customers + 1):

        # Generate random demand for the customer
        demand = random.randint(1, 10) * 10
        
        # Generate random coordinates for the customer
        x_coord = random.uniform(0, 100)
        y_coord = random.uniform(0, 100)

        # Generate random ready time for the customer
        ready_time = random.randint(0, 1000)

        # Generate random due date for the customer
        due_date = ready_time + random.randint(100, 300)

        # Create a Customer object and add it to the list of customers
        customers.append(Customer(i, x_coord, y_coord, demand, ready_time, due_date, service_time))

    # Generate the depot (warehouse) at the center of the customers
    depot = Customer(0, sum(customer.x_coord for customer in customers) / num_customers, sum(customer.y_coord for customer in customers) / num_customers, 0, 0, max(x.due_date for x in customers) + service_time + 100, 0)

    return [depot] + customers, truck_count, capacity

In [6]:
def read_dataset(file_path):
    # Reading the vehicle data (lines 3-4)
    vehicle_df = pd.read_csv(file_path, skiprows=4, nrows=1, sep='\\s+', names=['Number', 'Capacity'])
    
    # Reading the customer data (after line 9)
    customers_df = pd.read_csv(file_path, skiprows=9, sep='\\s+', names=['Cust No.', 'XCoord.', 'YCoord.', 'Demand', 'Ready Time', 'Due Date', 'Service Time'])

    truck_count = vehicle_df['Number'][0]  # Get the number of trucks
    capacity = vehicle_df['Capacity'][0]  # Get the capacity of each truck

    # Create a list of Customer objects from the customer data
    customers = [Customer(row['Cust No.'], row['XCoord.'], row['YCoord.'], row['Demand'], row['Ready Time'], row['Due Date'], row['Service Time']) for index, row in customers_df.iterrows()]
    
    return customers, truck_count, capacity

# if __name__ == '__main__':
#     for i in range(101, 113):
#         customers, warehouse, truck_count, capacity = read_dataset(f'dataset/c{i}.txt') # Read the dataset from the input file
#     
#         print("-" * 100)
#         print(f"Dataset: c{i}.txt")
#         print("-" * 100)
#         # Print the vehicle data
#         print(f"Number of trucks: {truck_count}")
#         print(f"Capacity of each truck: {capacity}")
#         print(f"Number of customers: {len(customers) - 1}")
#     
#         # Initialize the Ant Colony Optimization (ACO) algorithm
#         aco = ACO(
#             customers,
#             warehouse,
#             truck_count=truck_count,
#             truck_capacity=capacity,
#             iterations=100,
#             ants_count=15,
#             pheromone_importance=2.0,
#             heuristic_importance=2.0,
#             evaporation_rate=0.1,
#             pheromone_init=5.0,
#             debug=False
#         )
#     
#         # Run the ACO algorithm to find the best solution
#         best_solution, best_cost = aco.run()
#         
#         best_solution = aco.remove_unused_trucks(best_solution)  # Remove unused trucks from the solution
#         
#         # Plot the routes of the best solution
#         plot_routes(best_solution, warehouse)
#         
#         # Print the cost history
#         print_costs_history(ACO.costs)
#         
#         # Print the truck usage details
#         print_truck_usage(best_solution, warehouse, show_graphics=False)

if __name__ == '__main__':
    customers, truck_count, capacity = read_dataset('dataset/c1_2_1.txt')  # Read the dataset from the input file
    # customers, truck_count, capacity = instance_generator(10)  # Generate a random instance of the vehicle routing problem

    # Print the vehicle data
    print(f"Number of trucks: {truck_count}")
    print(f"Capacity of each truck: {capacity}")
    print(f"Number of customers: {len(customers) - 1}")

    # Initialize the Ant Colony Optimization (ACO) algorithm
    aco = ACO(
        customers,
        truck_count=truck_count,
        truck_capacity=capacity,
        iterations=75,
        ants_count=30,
        pheromone_importance=2.0,
        heuristic_importance=2.0,
        evaporation_rate=0.1,
        pheromone_init=5.0,
        debug=True
    )

    # Run the ACO algorithm to find the best solution
    best_solution, best_cost = aco.run()
    # print("Best solution:", best_solution)  # Print the best solution found
    # print("Best cost:", best_cost)  # Print the cost of the best solution

    best_solution = aco.remove_unused_trucks(best_solution)  # Remove unused trucks from the solution

    # Plot the routes of the best solution
    pr.plot_routes(best_solution, warehouse)

    # Print the cost history
    pr.print_costs_history(aco.costs)

    # Print the truck usage details
    pr.print_truck_usage(best_solution, warehouse)

Number of trucks: 50
Capacity of each truck: 200
Number of customers: 199
Running ACO...


IndexError: index 200 is out of bounds for axis 0 with size 200